In [ ]:
from itertools import combinations
from scipy.stats import spearmanr
from haversine import haversine
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

In [ ]:
files = []

for i in range(2020,2023):
    for j in range(12):
        files.append("traffic_density_{}".format(str(i) + str(j+1).zfill(2)))

for i in range(4):
    files.append("traffic_density_{}".format("2023" + str(i+1).zfill(2)))

In [ ]:
def get_distance(row, distances):
    return distances[row["District Pair"]]

In [ ]:
for file in files:
    df = pd.read_csv(file + ".csv")
    corr_df = pd.read_csv(file + "corr.csv")

    coordinates_df = df.groupby("GEOHASH")[["LONGITUDE", "LATITUDE"]].agg(lambda x: np.unique(x).tolist())
    coordinates_dict = coordinates_df.to_dict('index')

    coordinates_dict

    all_combs = np.array(list(combinations(list(np.unique(list(df["GEOHASH"]))), 2)))

    distances = {}

    for comb in tqdm(all_combs):
        c1 = (coordinates_dict[comb[0]]["LATITUDE"][0], coordinates_dict[comb[0]]["LONGITUDE"][0])
        c2 = (coordinates_dict[comb[1]]["LATITUDE"][0], coordinates_dict[comb[1]]["LONGITUDE"][0])
        d = haversine(c1, c2)
        distances[str((comb[0], comb[1]))] = d

    corr_df["Distance"] = corr_df.apply(get_distance, args=(distances,), axis=1)

    corr_df.to_csv(file + "_corr_distance.csv", index = False)
    print(file + "_corr_distance.csv finished!")